In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import re

In [9]:
# ChromeDriver 자동 설치 및 실행
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # 백그라운드 실행
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

# z-libaray korea book 웹사이트 열기 
driver.get("https://z-library.sk/category/14/Fiction/s/?languages%5B0%5D=korean&extensions%5B0%5D=AZW&extensions%5B1%5D=AZW3&extensions%5B2%5D=EPUB&extensions%5B3%5D=MOBI&extensions%5B4%5D=RTF&extensions%5B5%5D=TXT&extensions%5B6%5D=LIT&extensions%5B7%5D=FB2&extensions%5B8%5D=DJVU&extensions%5B9%5D=DJV&extensions%5B10%5D=CBZ&page=1")
print("ChromeDriver 실행 성공!")

# 웹 요소가 상호 작용 가능한 상태가 될 때까지 최대 20초 동안 기다림
wait = WebDriverWait(driver, 20)



ChromeDriver 실행 성공!


In [10]:
def download_book(title):
    """
    책 제목을 클릭하여 다운로드하는 함수
    """
    try:
        # 책 제목 클릭
        title_element = wait.until(
            EC.element_to_be_clickable((By.XPATH, f"//div[@slot='title' and contains(text(), '{title}')]"))
        )
        title_element.click()
        print(f"책 제목 '{title}' 클릭 성공! 상세 페이지로 이동 중...")

        # 상세 페이지 URL 확인
        current_url = driver.current_url
        print(f"현재 상세 페이지 URL: {current_url}")

        # 다운로드 옵션 버튼 클릭
        download_button = wait.until(
            EC.element_to_be_clickable((By.ID, "btnCheckOtherFormats"))
        )
        download_button.click()
        print(f"'{title}' 다운로드 옵션 버튼 클릭 성공!")

        # TXT 다운로드 버튼 클릭
        txt_download_button = wait.until(
            EC.element_to_be_clickable((By.XPATH, "//a[@data-convert_to='txt']"))
        )
        txt_download_button.click()
        print(f"'{title}' TXT 다운로드 버튼 클릭 성공! 다운로드 시작 중...")

        # 다운로드 완료 대기 (파일이 다운로드될 때까지 잠시 대기)
        time.sleep(30)  # 충분한 대기 시간 설정 (필요시 더 길게 조정)
    
    except Exception as e:
        print(f"오류 발생: {e}")
    finally:
        # 뒤로 가기
        driver.back()
        time.sleep(2)  # 페이지 로딩 대기

def go_to_next_page():
    """
    페이지 번호를 클릭하여 다음 페이지로 이동하는 함수
    """
    try:
        next_page_button = wait.until(
            EC.element_to_be_clickable((By.XPATH, "//a[contains(text(), 'Next')]"))
        )
        next_page_button.click()
        print("다음 페이지로 이동 중...")
        time.sleep(3)  # 페이지 로딩 대기
    except Exception as e:
        print(f"오류 발생: {e}")

def download_books_on_page():
    """
    현재 페이지에서 모든 책을 다운로드하는 함수
    """
    # 모든 책 제목을 가져옵니다
    book_titles = wait.until(
        EC.presence_of_all_elements_located((By.XPATH, "//div[@slot='title']"))
    )
    for title_element in book_titles:
        title = title_element.text
        print(f"책 제목: {title}")
        download_book(title)

def main():
    """
    책을 다운로드하는 메인 함수
    """
    page_number = 1

    while True:
        print(f"현재 {page_number} 페이지에서 책 다운로드 시작...")

        # 현재 페이지에서 책 다운로드
        download_books_on_page()

        # 50권이 다운로드되었으면, 다음 페이지로 이동
        try:
            go_to_next_page()
            page_number += 1
        except Exception as e:
            print(f"더 이상 페이지가 없습니다. 다운로드 완료!")
            break

    # 드라이버 종료
    #driver.quit()

if __name__ == "__main__":
    main()

현재 1 페이지에서 책 다운로드 시작...
책 제목: 아몬드(양장본 HardCover)
책 제목 '아몬드(양장본 HardCover)' 클릭 성공! 상세 페이지로 이동 중...
현재 상세 페이지 URL: https://z-library.sk/book/12032307/785d65/%EC%95%84%EB%AA%AC%EB%93%9C%EC%96%91%EC%9E%A5%EB%B3%B8-hardcover.html
'아몬드(양장본 HardCover)' 다운로드 옵션 버튼 클릭 성공!
'아몬드(양장본 HardCover)' TXT 다운로드 버튼 클릭 성공! 다운로드 시작 중...
책 제목: 창문 넘어 도망 친 100세 노인
책 제목 '창문 넘어 도망 친 100세 노인' 클릭 성공! 상세 페이지로 이동 중...
현재 상세 페이지 URL: https://z-library.sk/book/12033299/cee40f/%EC%B0%BD%EB%AC%B8-%EB%84%98%EC%96%B4-%EB%8F%84%EB%A7%9D-%EC%B9%9C-100%EC%84%B8-%EB%85%B8%EC%9D%B8.html
'창문 넘어 도망 친 100세 노인' 다운로드 옵션 버튼 클릭 성공!
'창문 넘어 도망 친 100세 노인' TXT 다운로드 버튼 클릭 성공! 다운로드 시작 중...
책 제목: 불편한 편의점 1
책 제목 '불편한 편의점 1' 클릭 성공! 상세 페이지로 이동 중...
현재 상세 페이지 URL: https://z-library.sk/book/23988613/93c9e6/%EB%B6%88%ED%8E%B8%ED%95%9C-%ED%8E%B8%EC%9D%98%EC%A0%90-1.html
'불편한 편의점 1' 다운로드 옵션 버튼 클릭 성공!
'불편한 편의점 1' TXT 다운로드 버튼 클릭 성공! 다운로드 시작 중...
책 제목: 노르웨이의 숲
책 제목 '노르웨이의 숲' 클릭 성공! 상세 페이지로 이동 중...
현재 상세 페이지 URL: https://z-library.sk/book/26346711/12ac

KeyboardInterrupt: 